# Day 12 - Vector operations

- https://adventofcode.com/2020/day/12

Moving our ship's position is a question of applying some vector operations: addition, multiplication, and rotation.

- The N, S, E & W instructions can be modeled with a unit vector (e.g. `(0, -1)` for _North_, with positive X and Y being East and South, respectively), multiplied with the count, being added to a position vector.
- R and L (rotation) can modeled by rotating a heading vector (again a unit vector pointing in one of the compass directions)
- F is multiplying the current heading then adding it to the position

I used Python OO techniques to model this, using type annotations throughout. The annotations are more generic than needed for part 1, but I neded to be able to extend this model for part 2.


In [1]:
from collections.abc import Iterable, Sequence
from dataclasses import dataclass, replace
from enum import Enum
from functools import reduce
from typing import Any, Generic, NamedTuple, Protocol, Type, TypeVar

T = TypeVar("T", bound="Position")
S = TypeVar("S", bound="BaseAction")


# an object that can implement the effect a navigation instruction has
# on a position
class ActionEffect(Protocol):
    def __call__(self, pos: T, value: int) -> T:
        pass


class Vector(NamedTuple):
    x: int = 0
    y: int = 0

    def __add__(self, other: Any) -> "Vector":
        if not isinstance(other, Vector):
            return NotImplemented
        return type(self)(self.x + other.x, self.y + other.y)

    def __mul__(self, factor: Any) -> "Vector":
        if not isinstance(factor, int):
            return NotImplemented
        return type(self)(self.x * factor, self.y * factor)

    def __matmul__(self, angle: Any) -> "Vector":
        if not isinstance(angle, int):
            return NotImplemented
        # simplified rotation calculations: only in steps of 90 degrees, so
        # no cos / sin functions needed.
        x, y = self
        fy, fx = (-1, 1) if angle > 0 else (1, -1)
        for _ in range(abs(angle) // 90):
            x, y = fy * y, fx * x
        return type(self)(x, y)

    def __abs__(self) -> int:
        return abs(self.x) + abs(self.y)


class Direction(Enum):
    north = Vector(0, -1)
    east = Vector(1, 0)
    south = Vector(0, 1)
    west = Vector(-1, 0)

    def __call__(self, pos: T, value: int) -> T:
        delta = self.value * value
        return replace(pos, pos=pos.pos + delta)


@dataclass(frozen=True)
class Position:
    pos: Vector = Vector()
    heading: Vector = Direction.east.value

    def apply(self: T, instr: Iterable["Instruction[S]"]) -> T:
        return reduce(lambda p, i: i(p), instr, self)

    @property
    def distance(self) -> int:
        return abs(self.pos)


class Rotation(Enum):
    left = -1
    right = 1

    def __call__(self, pos: T, value: int) -> T:
        heading = pos.heading @ (value * self.value)
        return replace(pos, heading=heading)


class Forward:
    def __call__(self, pos: T, value: int) -> T:
        return replace(pos, pos=pos.pos + (pos.heading * value))


class BaseAction(Enum):
    value: ActionEffect

    def __call__(self, pos: T, value: int) -> T:
        return self.value(pos, value)


class Action(BaseAction):
    N = Direction.north
    S = Direction.south
    E = Direction.east
    W = Direction.west
    L = Rotation.left
    R = Rotation.right
    F = Forward()


@dataclass(frozen=True)
class Instruction(Generic[S]):
    action: S
    value: int

    @classmethod
    def from_line(cls, line: str, action_class: Type[S]) -> "Instruction[S]":
        return cls(action_class[line[0]], int(line[1:]))

    def __call__(self, pos: T) -> T:
        return self.action(pos, self.value)


def parse_instructions(
    lines: Iterable[str], action_class=Action
) -> Sequence[Instruction[S]]:
    return [Instruction.from_line(line, action_class) for line in lines]


test = """\
F10
N3
F7
R90
F11
""".splitlines()

test_instructions: Iterable[Instruction[Action]] = parse_instructions(test)
assert Position().apply(test_instructions).distance == 25

In [2]:
import aocd

data = aocd.get_data(day=12, year=2020).splitlines()
instructions: Iterable[Instruction[Action]] = parse_instructions(data)

In [3]:
print("Part 1:", Position().apply(instructions).distance)

Part 1: 420


## Part 2 - refactoring

Instead of a unit vector for the heading, make the waypoint the heading. The only change we need to make then is how the N, S, E and W instructions are applied: to the heading instead of the position:


In [4]:
# Enum subclassing is... limited. I'd love to be able to subclass just to
# replace the __call__ implementation. But, alas. In addition, Mypy can't handle
# more advanced used of the Enum Functional API (python/mypy#4184), so we
# can't use a pre-defined `directions` structure either.
class WaypointDirection(Enum):
    north = Vector(0, -1)
    east = Vector(1, 0)
    south = Vector(0, 1)
    west = Vector(-1, 0)

    def __call__(self, pos: T, value: int) -> T:
        delta = self.value * value
        return replace(pos, heading=pos.heading + delta)


class WaypointAction(BaseAction):
    N = WaypointDirection.north
    S = WaypointDirection.south
    E = WaypointDirection.east
    W = WaypointDirection.west
    L = Rotation.left
    R = Rotation.right
    F = Forward()


def parse_waypoint_instructions(
    lines: Iterable[str],
) -> Iterable[Instruction[WaypointAction]]:
    return [Instruction.from_line(line, WaypointAction) for line in lines]


waypoint_start = Vector(10, -1)

test_wp = parse_waypoint_instructions(test)
assert Position(heading=waypoint_start).apply(test_wp).distance == 286

In [5]:
wp_instructions = parse_waypoint_instructions(data)
print("Part 2:", Position(heading=waypoint_start).apply(wp_instructions).distance)

Part 2: 42073
